In [163]:
from datetime import datetime, timedelta
from scrapingbee import ScrapingBeeClient
from opensky_api import OpenSkyApi
from opensky_api import OpenSkyStates
import pandas as pd
import requests
import random
import json
import time
import sys
import os
api = OpenSkyApi()


In [164]:
# OxyLabs Credentials
with open(f'{sys.path[0]}/keys/oxylabs.key', 'r') as f:
    creds = f.read().strip().split(' ')
    USERNAME = creds[0]
    PASSWORD = creds[1]

In [190]:
def get_states(USERNAME,PASSWORD):
    payload = {
        "source": "universal",
        "url": "https://opensky-network.org/api/states/all",
        "geo_location": "United Kingdom"
    }
    
    # Using Oxylabs to supply proxy IP Address to API Request
    response =  requests.request(
            "POST", "https://realtime.oxylabs.io/v1/queries",
            auth = (USERNAME,PASSWORD),
            json = payload
    )
    response = response.json()['results'][0]['content']
    return response

def json_to_df(response,date_selection):
    states=OpenSkyStates(json.loads(response))
    df = pd.DataFrame([vars(o) for o in states.states])
    df.to_csv(f'{os.path.dirname(sys.path[0])}/data/get_states/{date_selection}/csv/{datetime.now().isoformat()}.csv')    

def mk_directory_structure(date_selection):
    if os.path.exists(f'{os.path.dirname(sys.path[0])}/data/get_states/{date_selection}') == False:
        os.mkdir(f'{os.path.dirname(sys.path[0])}/data/get_states/{date_selection}')
        os.mkdir(f'{os.path.dirname(sys.path[0])}/data/get_states/{date_selection}/csv')
        os.mkdir(f'{os.path.dirname(sys.path[0])}/data/get_states/{date_selection}/images')
        os.mkdir(f'{os.path.dirname(sys.path[0])}/data/get_states/{date_selection}/images/eu')
        os.mkdir(f'{os.path.dirname(sys.path[0])}/data/get_states/{date_selection}/images/us')

In [191]:
date_selection = datetime.today().strftime('%Y-%m-%d') 
mk_directory_structure(date_selection)
for i in range(1000):
    try:
        response = get_states(USERNAME,PASSWORD)
        json_to_df(response,date_selection)
        print(f'State {i} | Saved on: {datetime.now().isoformat()} ...')
        time.sleep(60)
    except:
        print(f'State {i} | Returned Error ...')
        time.sleep(10)

State 0 | Saved on: 2024-05-05T20:14:36.741960 ...
